In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

data=pd.read_excel("赛题1测试数据集\融资信息.xlsx")
data=data.replace('未披露',np.nan)
data

,企业编号,融资日期,轮次,投资金额
0,4001,2011-05-05,IPO,NaN
1,4002,2011-05-05,IPO,NaN
2,4003,2011-05-11,IPO,NaN
3,4004,2011-05-11,IPO,NaN
4,4005,2011-05-11,IPO,NaN
...,...,...,...,...
964,4525,2006-09-26,IPO,NaN
965,4525,2018-10-16,IPO,593000万人民币
966,4525,2012-01-01,战略投资,NaN
967,4525,2016-03-01,战略投资,NaN


In [2]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

df=data.set_index("企业编号")
df1 = df[['融资日期']].applymap(to_day).groupby('企业编号').mean().T
df1 = df1.reindex(columns=list(range(4001,4526)))
df1

企业编号,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,...,4516,4517,4518,4519,4520,4521,4522,4523,4524,4525
融资日期,-2861.0,-2861.0,-2855.0,-2855.0,-2855.0,-2848.0,-2848.0,-2841.0,-2841.0,-1827.75,...,-2664.666667,-796.0,-696.0,-975.166667,-116.0,-95.0,-606.5,-67.0,-517.0,-3387.375


In [3]:
data=data.set_index("轮次").loc[["IPO","战略投资","A轮","上市","主板定向增发",
                          "B轮","天使轮","天使轮","股权转让","被收购","C轮","新三板"],:].reset_index()
for i in range(len(data)):
    if ( type(data.iloc[i,3])==np.str and data.iloc[i,3][0]!="数"):
        if(data.iloc[i,3][-1]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-1])
        elif(data.iloc[i,3][-2]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-2])
        elif(data.iloc[i,3][-3]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-3])
        elif(data.iloc[i,3][-4]=="万"):
             data.iloc[i,3]=float(data.iloc[i,3][:-4])
    elif (type(data.iloc[i,3])==np.str and data.iloc[i,3][0]=="数"):
            data.iloc[i,3]=np.nan
data 

,轮次,企业编号,融资日期,投资金额
0,IPO,4001,2011-05-05,NaN
1,IPO,4002,2011-05-05,NaN
2,IPO,4003,2011-05-11,NaN
3,IPO,4004,2011-05-11,NaN
4,IPO,4005,2011-05-11,NaN
...,...,...,...,...
966,新三板,4506,2014-08-29,NaN
967,新三板,4507,2015-10-15,NaN
968,新三板,4508,2015-07-15,NaN
969,新三板,4517,2015-12-18,NaN


In [4]:
data=data.drop(columns="融资日期").set_index('企业编号')
result=data.groupby(['企业编号','轮次']).sum().unstack().T.reindex(columns=list(range(4001,4526)))
result

企业编号       4001  4002  4003  4004  4005  4006  4007  4008     4009     4010  \
     轮次                                                                       
投资金额 A轮     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN      0.0   
     B轮     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN      NaN   
     C轮     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN      NaN   
     IPO    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0      NaN      0.0   
     上市     NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  45000.0      NaN   
...         ...   ...   ...   ...   ...   ...   ...   ...      ...      ...   
     天使轮    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN      NaN   
     战略投资   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN      0.0   
     新三板    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN      NaN   
     股权转让   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN  61800.0   
     被收购    NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN      NaN   

企业编号         ...     4516     4517  4518      4519  4520     4521     4522  \
     轮次      ...                                                             
投资金额 A轮      ...      0.0      NaN   NaN  300000.0   NaN      NaN      NaN   
     B轮      ...      NaN      NaN   NaN  800000.0   NaN      NaN      NaN   
     C轮      ...      NaN      NaN   NaN       NaN   NaN      NaN      NaN   
     IPO     ...      0.0  22400.0   0.0  546200.0   0.0  37700.0  31900.0   
     上市      ...      NaN      NaN   NaN       NaN   NaN      NaN      NaN   
...          ...      ...      ...   ...       ...   ...      ...      ...   
     天使轮     ...      0.0      NaN   NaN       NaN   NaN      NaN      0.0   
     战略投资    ...      NaN      NaN   NaN  134000.0   NaN      NaN      NaN   
     新三板     ...      NaN      0.0   0.0       NaN   NaN      NaN      NaN   
     股权转让    ...      NaN      NaN   NaN       NaN   NaN      NaN      NaN   
     被收购     ...      NaN      NaN   NaN       NaN   NaN      NaN      NaN   

企业编号          4523  4524      4525  
     轮次                             
投资金额 A轮        NaN   0.0     600.0  
     B轮        NaN   NaN    2780.0  
     C轮        NaN   NaN       NaN  
     IPO   45000.0   0.0  593000.0  
     上市        NaN   NaN       NaN  
...            ...   ...       ...  
     天使轮       NaN   NaN       NaN  
     战略投资      NaN   NaN   26000.0  
     新三板       NaN   NaN       NaN  
     股权转让      NaN   NaN       NaN  
     被收购       NaN   NaN       NaN  

[11 rows x 525 columns]

In [5]:
result.to_pickle('融资信息-testing.pickle')